CER033 - Sign Controller certificate with cluster Root CA
=========================================================

This notebook signs the certificate created using:

-   [CER023 - Create Controller
    certificate](../cert-management/cer023-create-controller-cert.ipynb)

with the generate Root CA Certificate, created using either:

-   [CER001 - Generate a Root CA
    certificate](../cert-management/cer001-create-root-ca.ipynb)
-   [CER003 - Upload existing Root CA
    certificate](../cert-management/cer003-upload-existing-root-ca.ipynb)

Steps
-----

### Parameters

In [ ]:
import getpass

app_name = "controller"
scaledset_name = "control"
container_name = "controller"
prefix_keyfile_name = "controller"
common_name = "controller-svc"

country_name = "US"
state_or_province_name = "Illinois"
locality_name = "Chicago"
organization_name = "Contoso"
organizational_unit_name = "Finance"
email_address = f"{getpass.getuser()}@contoso.com"

ssl_configuration_file = "ca.openssl.cnf"

days = "825" # the number of days to certify the certificate for

certificate_filename = "cluster-ca-certificate.crt"
private_key_filename = "cluster-ca-privatekey.pem"

test_cert_store_root = "/var/opt/secrets/test-certificates"

### Common functions

Define helper functions used in this notebook.

In [ ]:
# Define `run` function for transient fault handling, suggestions on error, and scrolling updates on Windows
import sys
import os
import re
import json
import platform
import shlex
import shutil
import datetime

from subprocess import Popen, PIPE
from IPython.display import Markdown

retry_hints = {}
error_hints = {}
install_hint = {}

first_run = True
rules = None

def run(cmd, return_output=False, no_output=False, retry_count=0):
    """
    Run shell command, stream stdout, print stderr and optionally return output
    """
    MAX_RETRIES = 5
    output = ""
    retry = False

    global first_run
    global rules

    if first_run:
        first_run = False
        rules = load_rules()

    # shlex.split is required on bash and for Windows paths with spaces
    #
    cmd_actual = shlex.split(cmd)

    # Store this (i.e. kubectl, python etc.) to support binary context aware error_hints and retries
    #
    user_provided_exe_name = cmd_actual[0].lower()

    # When running python, use the python in the ADS sandbox ({sys.executable})
    #
    if cmd.startswith("python "):
        cmd_actual[0] = cmd_actual[0].replace("python", sys.executable)

        # On Mac, when ADS is not launched from terminal, LC_ALL may not be set, which causes pip installs to fail
        # with:
        #
        #    UnicodeDecodeError: 'ascii' codec can't decode byte 0xc5 in position 4969: ordinal not in range(128)
        #
        # Setting it to a default value of "en_US.UTF-8" enables pip install to complete
        #
        if platform.system() == "Darwin" and "LC_ALL" not in os.environ:
            os.environ["LC_ALL"] = "en_US.UTF-8"

    # To aid supportabilty, determine which binary file will actually be executed on the machine
    #
    which_binary = None

    # Special case for CURL on Windows.  The version of CURL in Windows System32 does not work to
    # get JWT tokens, it returns "(56) Failure when receiving data from the peer".  If another instance
    # of CURL exists on the machine use that one.  (Unfortunately the curl.exe in System32 is almost
    # always the first curl.exe in the path, and it can't be uninstalled from System32, so here we
    # look for the 2nd installation of CURL in the path)
    if platform.system() == "Windows" and cmd.startswith("curl "):
        path = os.getenv('PATH')
        for p in path.split(os.path.pathsep):
            p = os.path.join(p, "curl.exe")
            if os.path.exists(p) and os.access(p, os.X_OK):
                if p.lower().find("system32") == -1:
                    cmd_actual[0] = p
                    which_binary = p
                    break

    # Find the path based location (shutil.which) of the executable that will be run (and display it to aid supportability), this
    # seems to be required for .msi installs of azdata.cmd/az.cmd.  (otherwise Popen returns FileNotFound) 
    #
    # NOTE: Bash needs cmd to be the list of the space separated values hence shlex.split.
    #
    if which_binary == None:
        which_binary = shutil.which(cmd_actual[0])

    if which_binary == None:
        if user_provided_exe_name in install_hint and install_hint[user_provided_exe_name] is not None:
            display(Markdown(f'HINT: Use [{install_hint[user_provided_exe_name][0]}]({install_hint[user_provided_exe_name][1]}) to resolve this issue.'))

        raise FileNotFoundError(f"Executable '{cmd_actual[0]}' not found in path (where/which)")
    else:   
        cmd_actual[0] = which_binary

    start_time = datetime.datetime.now().replace(microsecond=0)

    print(f"START: {cmd} @ {start_time} ({datetime.datetime.utcnow().replace(microsecond=0)} UTC)")
    print(f"       using: {which_binary} ({platform.system()} {platform.release()} on {platform.machine()})")
    print(f"       cwd: {os.getcwd()}")

    # Command-line tools such as CURL and AZDATA HDFS commands output
    # scrolling progress bars, which causes Jupyter to hang forever, to
    # workaround this, use no_output=True
    #

    # Work around a infinite hang when a notebook generates a non-zero return code, break out, and do not wait
    #
    wait = True 

    try:
        if no_output:
            p = Popen(cmd_actual)
        else:
            p = Popen(cmd_actual, stdout=PIPE, stderr=PIPE, bufsize=1)
            with p.stdout:
                for line in iter(p.stdout.readline, b''):
                    line = line.decode()
                    if return_output:
                        output = output + line
                    else:
                        if cmd.startswith("azdata notebook run"): # Hyperlink the .ipynb file
                            regex = re.compile('  "(.*)"\: "(.*)"') 
                            match = regex.match(line)
                            if match:
                                if match.group(1).find("HTML") != -1:
                                    display(Markdown(f' - "{match.group(1)}": "{match.group(2)}"'))
                                else:
                                    display(Markdown(f' - "{match.group(1)}": "[{match.group(2)}]({match.group(2)})"'))

                                    wait = False
                                    break # otherwise infinite hang, have not worked out why yet.
                        else:
                            print(line, end='')
                            if rules is not None:
                                apply_expert_rules(line)

        if wait:
            p.wait()
    except FileNotFoundError as e:
        if install_hint is not None:
            display(Markdown(f'HINT: Use {install_hint} to resolve this issue.'))

        raise FileNotFoundError(f"Executable '{cmd_actual[0]}' not found in path (where/which)") from e

    exit_code_workaround = 0 # WORKAROUND: azdata hangs on exception from notebook on p.wait()

    if not no_output:
        for line in iter(p.stderr.readline, b''):
            line_decoded = line.decode()

            # azdata emits a single empty line to stderr when doing an hdfs cp, don't
            # print this empty "ERR:" as it confuses.
            #
            if line_decoded == "":
                continue
            
            print(f"STDERR: {line_decoded}", end='')

            if line_decoded.startswith("An exception has occurred") or line_decoded.startswith("ERROR: An error occurred while executing the following cell"):
                exit_code_workaround = 1

            if user_provided_exe_name in error_hints:
                for error_hint in error_hints[user_provided_exe_name]:
                    if line_decoded.find(error_hint[0]) != -1:
                        display(Markdown(f'HINT: Use [{error_hint[1]}]({error_hint[2]}) to resolve this issue.'))

            if rules is not None:
                apply_expert_rules(line_decoded)

            if user_provided_exe_name in retry_hints:
                for retry_hint in retry_hints[user_provided_exe_name]:
                    if line_decoded.find(retry_hint) != -1:
                        if retry_count < MAX_RETRIES:
                            print(f"RETRY: {retry_count} (due to: {retry_hint})")
                            retry_count = retry_count + 1
                            output = run(cmd, return_output=return_output, retry_count=retry_count)

                            if return_output:
                                return output
                            else:
                                return

    elapsed = datetime.datetime.now().replace(microsecond=0) - start_time

    # WORKAROUND: We avoid infinite hang above in the `azdata notebook run` failure case, by inferring success (from stdout output), so
    # don't wait here, if success known above
    #
    if wait: 
        if p.returncode != 0:
            raise SystemExit(f'Shell command:\n\n\t{cmd} ({elapsed}s elapsed)\n\nreturned non-zero exit code: {str(p.returncode)}.\n')
    else:
        if exit_code_workaround !=0 :
            raise SystemExit(f'Shell command:\n\n\t{cmd} ({elapsed}s elapsed)\n\nreturned non-zero exit code: {str(exit_code_workaround)}.\n')


    print(f'\nSUCCESS: {elapsed}s elapsed.\n')

    if return_output:
        return output

def load_json(filename):
    with open(filename, encoding="utf8") as json_file:
        return json.load(json_file)

def load_rules():

    try:

        # Load this notebook as json to get access to the expert rules in the notebook metadata.
        #
        j = load_json("cer033-sign-controller-generated-cert.ipynb")

    except:
        pass # If the user has renamed the book, we can't load ourself.  NOTE: Is there a way in Jupyter, to know your own filename?

    else:
        if "metadata" in j and \
            "azdata" in j["metadata"] and \
            "expert" in j["metadata"]["azdata"] and \
            "rules" in j["metadata"]["azdata"]["expert"]:

            rules = j["metadata"]["azdata"]["expert"]["rules"]

            rules.sort() # Sort rules, so they run in priority order (the [0] element).  Lowest value first.

            # print (f"EXPERT: There are {len(rules)} rules to evaluate.")

            return rules

def apply_expert_rules(line):

    global rules

    for rule in rules:

        # rules that have 9 elements are the injected (output) rules (the ones we want).  Rules
        # with only 8 elements are the source (input) rules, which are not expanded (i.e. TSG029,
        # not ../repair/tsg029-nb-name.ipynb)
        if len(rule) == 9:
            notebook = rule[1]
            cell_type = rule[2]
            output_type = rule[3] # i.e. stream or error
            output_type_name = rule[4] # i.e. ename or name 
            output_type_value = rule[5] # i.e. SystemExit or stdout
            details_name = rule[6]  # i.e. evalue or text 
            expression = rule[7].replace("\\*", "*") # Something escaped *, and put a \ in front of it!

            # print(f"EXPERT: If rule '{expression}' satisfied', run '{notebook}'.")

            if re.match(expression, line, re.DOTALL):

                # print("EXPERT: MATCH: name = value: '{0}' = '{1}' matched expression '{2}', therefore HINT '{4}'".format(output_type_name, output_type_value, expression, notebook))

                match_found = True

                display(Markdown(f'HINT: Use [{notebook}]({notebook}) to resolve this issue.'))



print('Common functions defined successfully.')

# Hints for binary (transient fault) retry, (known) error and install guide
#
retry_hints = {'kubectl': ['A connection attempt failed because the connected party did not properly respond after a period of time, or established connection failed because connected host has failed to respond']}
error_hints = {'kubectl': [['no such host', 'TSG010 - Get configuration contexts', '../monitor-k8s/tsg010-get-kubernetes-contexts.ipynb'], ['no such host', 'TSG011 - Restart sparkhistory server', '../repair/tsg011-restart-sparkhistory-server.ipynb'], ['No connection could be made because the target machine actively refused it', 'TSG056 - Kubectl fails with No connection could be made because the target machine actively refused it', '../repair/tsg056-kubectl-no-connection-could-be-made.ipynb']]}
install_hint = {'kubectl': ['SOP036 - Install kubectl command line interface', '../install/sop036-install-kubectl.ipynb']}

### Get the Kubernetes namespace for the big data cluster

Get the namespace of the big data cluster use the kubectl command line
interface .

NOTE: If there is more than one big data cluster in the target
Kubernetes cluster, then set \[0\] to the correct value for the big data
cluster.

In [ ]:
# Place Kubernetes namespace name for BDC into 'namespace' variable

try:
    namespace = run(f'kubectl get namespace --selector=MSSQL_CLUSTER -o jsonpath={{.items[0].metadata.name}}', return_output=True)
except:
    from IPython.display import Markdown
    print(f"ERROR: Unable to find a Kubernetes namespace with label 'MSSQL_CLUSTER'.  SQL Server Big Data Cluster Kubernetes namespaces contain the label 'MSSQL_CLUSTER'.")
    display(Markdown(f'HINT: Use [TSG081 - Get namespaces (Kubernetes)](../monitor-k8s/tsg081-get-kubernetes-namespaces.ipynb) to resolve this issue.'))
    display(Markdown(f'HINT: Use [TSG010 - Get configuration contexts](../monitor-k8s/tsg010-get-kubernetes-contexts.ipynb) to resolve this issue.'))
    display(Markdown(f'HINT: Use [SOP011 - Set kubernetes configuration context](../common/sop011-set-kubernetes-context.ipynb) to resolve this issue.'))
    raise
else:
    print(f'The SQL Server Big Data Cluster Kubernetes namespace is: {namespace}')

### Create a temporary directory to stage files

In [ ]:
# Create a temporary directory to hold configuration files

import tempfile

temp_dir = tempfile.mkdtemp()

print(f"Temporary directory created: {temp_dir}")

### Helper function to save configuration files to disk

In [ ]:
# Define helper function 'save_file' to save configuration files to the temporary directory created above
import os
import io

def save_file(filename, contents):
    with io.open(os.path.join(temp_dir, filename), "w", encoding='utf8', newline='\n') as text_file:
      text_file.write(contents)

      print("File saved: " + os.path.join(temp_dir, filename))

### Get name of the ‘Running’ `controller` `pod`

In [ ]:
# Place the name  of the 'Running' controller pod in variable `controller`

controller = run(f'kubectl get pod --selector=app=controller -n {namespace} -o jsonpath={{.items[0].metadata.name}} --field-selector=status.phase=Running', return_output=True)

print(f"Controller pod name: {controller}")

### Create Signing Request configuration file

In [ ]:
certificate = f"""
[ ca ]
default_ca    = CA_default      # The default ca section

[ CA_default ]
default_days     = 1000         # How long to certify for
default_crl_days = 30           # How long before next CRL
default_md       = sha256       # Use public key default MD
preserve         = no           # Keep passed DN ordering

x509_extensions = ca_extensions # The extensions to add to the cert

email_in_dn     = no            # Don't concat the email in the DN
copy_extensions = copy          # Required to copy SANs from CSR to cert

base_dir      = {test_cert_store_root}
certificate   = $base_dir/{certificate_filename}   # The CA certifcate
private_key   = $base_dir/{private_key_filename}    # The CA private key
new_certs_dir = $base_dir              # Location for new certs after signing
database      = $base_dir/index.txt    # Database index file
serial        = $base_dir/serial.txt   # The current serial number

unique_subject = no  # Set to 'no' to allow creation of
                     # several certificates with same subject.

[ req ]
default_bits       = 2048
default_keyfile    = {test_cert_store_root}/{private_key_filename}
distinguished_name = ca_distinguished_name
x509_extensions    = ca_extensions
string_mask        = utf8only

[ ca_distinguished_name ]
countryName         = Country Name (2 letter code)
countryName_default = {country_name}

stateOrProvinceName         = State or Province Name (full name)
stateOrProvinceName_default = {state_or_province_name}

localityName                = Locality Name (eg, city)
localityName_default        = {locality_name}

organizationName            = Organization Name (eg, company)
organizationName_default    = {organization_name}

organizationalUnitName         = Organizational Unit (eg, division)
organizationalUnitName_default = {organizational_unit_name}

commonName         = Common Name (e.g. server FQDN or YOUR name)
commonName_default = {common_name}

emailAddress         = Email Address
emailAddress_default = {email_address}

[ ca_extensions ]
subjectKeyIdentifier   = hash
authorityKeyIdentifier = keyid:always, issuer
basicConstraints       = critical, CA:true
keyUsage               = keyCertSign, cRLSign

[ signing_policy ]
countryName            = optional
stateOrProvinceName    = optional
localityName           = optional
organizationName       = optional
organizationalUnitName = optional
commonName             = supplied
emailAddress           = optional

[ signing_req ]
subjectKeyIdentifier   = hash
authorityKeyIdentifier = keyid,issuer
basicConstraints       = CA:FALSE
keyUsage               = digitalSignature, keyEncipherment
"""

save_file(ssl_configuration_file, certificate)


### Copy certificate configuration to `controller` `pod`

In [ ]:
import os

cwd = os.getcwd()
os.chdir(temp_dir) # Use chdir to workaround kubectl bug on Windows, which incorrectly processes 'c:\' on kubectl cp cmd line 

run(f'kubectl exec {controller} -c controller -n {namespace} -- bash -c "mkdir -p {test_cert_store_root}/{app_name}"')

run(f'kubectl cp {ssl_configuration_file} {controller}:{test_cert_store_root}/{app_name}/{ssl_configuration_file} -c controller -n {namespace}')

os.chdir(cwd)

### Obtain CA certificate

In [ ]:
cmd = f"cp /var/run/secrets/certificates/rootca/cluster-ca-certificate.crt {test_cert_store_root}/cluster-ca-certificate.crt"

run(f'kubectl exec {controller} -c controller -n {namespace} -- bash -c "{cmd}"')

### Extract the private key from pfx

In [ ]:
cmd = f"NO_PASSWORD= openssl pkcs12 -in /var/run/secrets/certificates/rootca/cluster-ca-certificate.p12 -out {test_cert_store_root}/{private_key_filename} -nocerts -nodes -password env:NO_PASSWORD"

run(f'kubectl exec {controller} -c controller -n {namespace} -- bash -c "{cmd}"')

### Set next serial number

In [ ]:
run(f'kubectl exec {controller} -n {namespace} -c controller -- bash -c "test -f {test_cert_store_root}/index.txt || touch {test_cert_store_root}/index.txt"')
run(f"""kubectl exec {controller} -n {namespace} -c controller -- bash -c "test -f {test_cert_store_root}/serial.txt || echo '00' > {test_cert_store_root}/serial.txt" """)

current_serial_number = run(f"""kubectl exec {controller} -n {namespace} -c controller -- bash -c "cat {test_cert_store_root}/serial.txt" """, return_output=True)

# The serial number is hex
new_serial_number = int(f"0x{current_serial_number}", 0) + 1

run(f"""kubectl exec {controller} -n {namespace} -c controller -- bash -c "echo '{new_serial_number:02X}' > {test_cert_store_root}/serial.txt" """)

### Create private key and certificate signing request

In [ ]:
cmd = f"openssl ca -batch -notext -config {test_cert_store_root}/{app_name}/ca.openssl.cnf -policy signing_policy -extensions signing_req -out {test_cert_store_root}/{app_name}/{prefix_keyfile_name}-certificate.pem -infiles {test_cert_store_root}/{app_name}/{prefix_keyfile_name}-signingrequest.csr"

run(f'kubectl exec {controller} -c controller -n {namespace} -- bash -c "{cmd}"')

### Display certificate

Use openssl x509 to display the certificate, so it can be visually
verified to be correct.

In [ ]:
cmd = f"openssl x509 -in {test_cert_store_root}/{app_name}/{prefix_keyfile_name}-certificate.pem -text -noout"

run(f'kubectl exec {controller} -c controller -n {namespace} -- bash -c "{cmd}"')

### Generate the `controller` certificate in PKCS12 format

For the controller certificate, generate the certificate in PKCS12
format.

In order to obtain the certificate in PKCS12 format from the generated
private key and certificate file above, run the following:

In [ ]:
cmd = f'NO_PASSWORD= openssl pkcs12 -export -out {test_cert_store_root}/{app_name}/{prefix_keyfile_name}-certificate.p12 -inkey {test_cert_store_root}/{app_name}/{prefix_keyfile_name}-privatekey.pem -in {test_cert_store_root}/{app_name}/{prefix_keyfile_name}-certificate.pem -password env:NO_PASSWORD'

run(f'kubectl exec {controller} -n {namespace} -c {app_name} -- bash -c "{cmd}"')

### Clean up the extracted controller private key

Remove the private key that was extract from the
cluster-ca-certificate.p12 above.

In [ ]:
cmd = f"rm {test_cert_store_root}/{private_key_filename}"

run(f'kubectl exec {controller} -c controller -n {namespace} -- bash -c "{cmd}"')

### Clean up temporary directory for staging configuration files

In [ ]:
# Delete the temporary directory used to hold configuration files

import shutil

shutil.rmtree(temp_dir)

print(f'Temporary directory deleted: {temp_dir}')

In [ ]:
print('Notebook execution complete.')

Related
-------

-   [CER040 - Install signed Management Proxy
    certificate](../cert-management/cer040-install-service-proxy-cert.ipynb)

-   [CER022 - Create App Proxy
    certificate](../cert-management/cer022-create-app-proxy-cert.ipynb)

-   [CER042 - Install signed App-Proxy
    certificate](../cert-management/cer042-install-app-proxy-cert.ipynb)